### 作用域

- 变量
    * 局部变量
    1-1 赋值之前引用了局部变量b=6 错 
    有全局变量声明b=5
    * 全局变量
    1-2 声明global
    
- 对象

- 闭包
        闭包是由函数与其相关的引用环境组合而成的实体
        引用环境，在执行运行的某个时间点，所有处于活跃状态的变量所组成的集合
        函数可以作为另外一个函数的返回值 或者 参数 ，还可以作为一个 变量的值
    * ? 闭包__code__
        对象调用
        

In [5]:
# 1-1
def func(a):
    print(a)
    print(b)
    b=6
    
b=5
func(10)

10


UnboundLocalError: local variable 'b' referenced before assignment

In [7]:
# 1-2
def func(a):
    global b
    print(a)
    print(b)
    b=30
    
b=20
func(10)
print(b)

10
20
30


In [14]:
class Averager(object):
    def __init__(self):
        self.series=[]
        
    def __call__(self,value):
        self.series.append(value)
        total=sum(self.series)
        return total/len(self.series) 
    
age=Averager()
print(age(10))
print(age(20))
print(age(30))
print(dir(age))

10.0
15.0
20.0
['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'series']


In [16]:
def make_averager():
    series=[]
    def averager(value):
        series.append(value)
        total=sum(series)
        return total/len(series)
    return averager

age=make_averager()
print(age(10))
print(age(20))
print(age(30))
print(dir(age))

print(age.__code__.co_varnames)
print(age.__code__.co_freevars)

10.0
15.0
20.0
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
('value', 'total')
('series',)


### 类

    * 对象的 or 类的 数据成员
        1-1 
* 方法
* 属性


In [2]:
# 1-1
class Demo(object):
    total=0
    
    def __new__(cls,*args,**kwargs):
        if cls.total>=2:
            raise Exception('最多')
        else:
            return object.__new__(cls)
        
    def __init__(self):
        Demo.total=Demo.total+1
t1=Demo()
t2=Demo()
t3=Demo()

Exception: 最多

### 协程

- yield
    * datum = yield,可以产出值,yield关键字后面没有表达式,生成器产出None
    * 调用方是通过send(datum)的方式把数据提供给协程使用
    * next(...)当激活生成器后，程序会停在yield这里,但并没有给x赋值
    * 协程在运行过程中有四个状态
    @ 1-1
        ```
            GEN_CREATE:等待开始执行
            GEN_RUNNING:解释器正在执行，这个状态一般看不到
            GEN_SUSPENDED:在yield表达式处暂停
            GEN_CLOSED:执行结束
        ```

- yield from
    @ 1-2

In [41]:
# 1-1
def simple_core(a):
    print('->start a=',a)
    b=yield a
    print('->received b=',b)
    c=yield a+b
    print('->received c=',c)
#     d=yield a+b+c
    
abc=simple_core(100)
from inspect import getgeneratorstate
getgeneratorstate(abc)#查看状态
next(abc)
getgeneratorstate(abc)
abc.send(15)
# getgeneratorstate(abc)
# abc.send(20)
# getgeneratorstate(abc)

->start a= 0
->received b= 15


15

In [44]:
data={
    'girls;kg':[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

from collections import namedtuple
Result=namedtuple('Result',['count','average'])

def averager():
    total=0.0
    count=0
    average=None
    while True:
        term=yield
        if term is None:
            break
        total+=term
        count+=1
        average=total/count
    return Result(count,average)

def grouper(results,key):
    while True:
        results[key]=yield from averager()
        
def report(results):
    for key,value in sorted(results.items()):
        group,unit=key.split(';')
        print('{} {} averageing--> {} {}'.format(value.count,group,value.average,unit))
        print('{:2} {:5} averageing--> {:.2f} {}'.format(value.count,group,value.average,unit))
        print('--------------------------------')

def main(data):
    results={}
    for key,values in data.items():
        group=grouper(results,key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)
        

if __name__=='__main__':
    main(data)